# [作業目標]
試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果
[作業重點]
仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6])

作業1
試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [71]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
import sklearn
sklearn.__version__
warnings.filterwarnings('ignore')

data_path = 'desktop/data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
print(df.head())
df.isnull().any()

   Pclass                                               Name     Sex   Age  \
0       3                            Braund, Mr. Owen Harris    male  22.0   
1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2       3                             Heikkinen, Miss. Laina  female  26.0   
3       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4       3                           Allen, Mr. William Henry    male  35.0   

   SibSp  Parch            Ticket     Fare Cabin Embarked  
0      1      0         A/5 21171   7.2500   NaN        S  
1      1      0          PC 17599  71.2833   C85        C  
2      0      0  STON/O2. 3101282   7.9250   NaN        S  
3      1      0            113803  53.1000  C123        S  
4      0      0            373450   8.0500   NaN        S  


Pclass      False
Name        False
Sex         False
Age          True
SibSp       False
Parch       False
Ticket      False
Fare        False
Cabin        True
Embarked     True
dtype: bool

作業1
試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )


In [72]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼(以Pclass對Age)做群聚編碼
"""
Your Code Here
"""

df['Pclass'] = df['Pclass'].fillna('None')
mean_df = df.groupby(['Pclass'])['Age'].mean().reset_index()
mode_df = df.groupby(['Pclass'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Pclass'])['Age'].median().reset_index()
max_df = df.groupby(['Pclass'])['Age'].max().reset_index()
count_df = df.groupby(['Pclass'])['Age'].count().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Pclass'])
temp = pd.merge(temp, median_df, how='left', on=['Pclass'])
temp = pd.merge(temp, max_df, how='left', on=['Pclass'])
temp = pd.merge(temp,count_df,how='left',on=['Pclass'])
temp.columns = ['Pclass', 'Pclass_age_Mean', 'Pclass_age_Mode', 'Pclass_age_Median', 'Pclass_age_Max','Pclass_age_count']
temp

,Pclass,Pclass_age_Mean,Pclass_age_Mode,Pclass_age_Median,Pclass_age_Max,Pclass_age_count
0,1,38.233441,35.0,37.0,80.0,186
1,2,29.877630,24.0,29.0,70.0,173
2,3,25.140620,22.0,24.0,74.0,355


In [73]:
df = pd.merge(df, temp, how='left', on=['Pclass'])
df = df.drop(['Pclass'] , axis=1)
df.head()

,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_age_Mean,Pclass_age_Mode,Pclass_age_Median,Pclass_age_Max,Pclass_age_count
0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,25.140620,22.0,24.0,74.0,355
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,38.233441,35.0,37.0,80.0,186
2,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,25.140620,22.0,24.0,74.0,355
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,38.233441,35.0,37.0,80.0,186
4,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,25.140620,22.0,24.0,74.0,355


In [74]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_age_Mean', 'Pclass_age_Mode', 'Pclass_age_Median', 'Pclass_age_Max', 'Pclass_age_count']



,Age,SibSp,Parch,Fare,Pclass_age_Mean,Pclass_age_Mode,Pclass_age_Median,Pclass_age_Max,Pclass_age_count
0,22.0,1,0,7.2500,25.140620,22.0,24.0,74.0,355
1,38.0,1,0,71.2833,38.233441,35.0,37.0,80.0,186
2,26.0,0,0,7.9250,25.140620,22.0,24.0,74.0,355
3,35.0,1,0,53.1000,38.233441,35.0,37.0,80.0,186
4,35.0,0,0,8.0500,25.140620,22.0,24.0,74.0,355


In [75]:
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Pclass_age_Mean', 'Pclass_age_Mode', 'Pclass_age_Median', 'Pclass_age_Max','Pclass_age_count']\
                   , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
print(f'LinearRehression Score:',cross_val_score(estimator, train_X, train_Y, cv=5).mean())
# 原始特徵 + 羅傑斯迴歸
estimator = LogisticRegression()
print(f'LogisticRegression Score:',cross_val_score(estimator, train_X, train_Y, cv=5).mean())


LinearRehression Score: 0.055727824174331086
LogisticRegression Score: 0.6408846483794697


作業2
將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
Ans: 有提升,但模型分數相較先前標籤編碼及雜湊編碼 約0.8 還有一段距離.

In [76]:
train_X = MMEncoder.fit_transform(df)
estimator = LinearRegression()
print(f'LinearRehression Score:',cross_val_score(estimator, train_X, train_Y, cv=5).mean())
estimator = LogisticRegression()
print(f'LogisticRegression Score:',cross_val_score(estimator, train_X, train_Y, cv=5).mean())


LinearRehression Score: 0.11804541822621831
LogisticRegression Score: 0.6926462176043793
